In [2]:
import pandas as pd

# READ DATASETS

In [3]:
folder = "./datasets/"
output_folder = "./datasets_models/"

### STATIONS DATASET


In [12]:
df_comarcas = pd.read_csv(folder + "t15902.csv", sep=";", header=0)

In [4]:
df_stations = pd.read_csv(folder + "DB_estaciones.txt", sep=";", header=0)

In [5]:

df_mesures = pd.read_csv(folder + "datos_precip.csv", header=0).drop(['Unnamed: 0'], axis=1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jairo\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jairo\AppData\Local\Temp\ipykernel_22164\3659156374.py", line 1, in <module>
    df_mesures = pd.read_csv(folder + "datos_precip.csv", header=0).drop(['Unnamed: 0'], axis=1)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jairo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\io\parsers\readers.py", line 912, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jairo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\io\parsers\readers.py", line 583, in _read
    return parser.read(nrows)
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\jairo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\io\parsers\readers.py", l

In [11]:
df_stations.comarca.unique()

array(['Alt Camp', 'Alt Empordà', 'Alt Penedès', 'Alt Urgell',
       'Alta Ribagorça', 'Anoia', 'Bages', 'Baix Camp', 'Baix Ebre',
       'Baix Empordà', 'Baix Llobregat', 'Baix Penedès', 'Barcelonès',
       'Berguedà', 'Cerdanya', 'Conca de Barberà', 'Garraf', 'Garrigues',
       'Garrotxa', 'Gironès', 'Maresme', 'Montsià', 'Noguera', 'Osona',
       'Pallars Jussà', 'Pallars Sobirà', "Pla d'Urgell",
       "Pla de l'Estany", 'Priorat', "Ribera d'Ebre", 'Ripollès',
       'Segarra', 'Segrià', 'Selva', 'Solsonès', 'Tarragonès',
       'Terra Alta', 'Urgell', "Val d'Aran", 'Vallès Occidental',
       'Vallès Oriental'], dtype=object)

In [13]:
df_comarcas.Comarca.unique()

array(['Alt Camp', 'Alt Empordà', 'Alt Penedès', 'Alt Urgell',
       'Alta Ribagorça', 'Anoia', 'Aran', 'Bages', 'Baix Camp',
       'Baix Ebre', 'Baix Empordà', 'Baix Llobregat', 'Baix Penedès',
       'Barcelonès', 'Berguedà', 'Cerdanya', 'Conca de Barberà', 'Garraf',
       'Garrigues', 'Garrotxa', 'Gironès', 'Maresme', 'Moianès',
       'Montsià', 'Noguera', 'Osona', 'Pallars Jussà', 'Pallars Sobirà',
       "Pla d'Urgell", "Pla de l'Estany", 'Priorat', "Ribera d'Ebre",
       'Ripollès', 'Segarra', 'Segrià', 'Selva', 'Solsonès', 'Tarragonès',
       'Terra Alta', 'Urgell', 'Vallès Occidental', 'Vallès Oriental',
       'Cataluña'], dtype=object)

# PROCESS DATA

In [7]:
codiE_por_comarca = {}
for comarca, group in df_stations.groupby('comarca'):
    codiE_por_comarca[comarca] = list(group['codiE'].unique())

In [8]:
comarcas_bcn = ['Vallès Oriental', 'Alt Penedès', 'Baix Llobregat', 'Barcelonès', 'Garraf', 'Maresme', 'Vallès Occidental']
codiE_comarca = [codiE_por_comarca[i] for i in comarcas_bcn]

In [9]:
data_comarca = []
for i in comarcas_bcn:
    df_filtrado = df_mesures[df_mesures['codi_estacio'].isin(codiE_por_comarca[i])]
    data_comarca.append(df_filtrado.reset_index(drop=True))

In [10]:
for comarca in data_comarca:
    comarca.data_lectura = comarca.data_lectura.str.split('T').str[0]

In [11]:
media_por_dia_comarca = []
for comarca in data_comarca:
    comarca['data_lectura'] = pd.to_datetime(comarca['data_lectura'])
    media_por_dia_comarca.append(comarca.groupby([comarca['data_lectura'].dt.date, 'codi_variable'])['valor_lectura'].mean().reset_index())

In [12]:
nuevo_orden = ["comarca", "codi_variable", "data_lectura", "valor_lectura"]
for i in range(len(media_por_dia_comarca)):
    media_por_dia_comarca[i]["comarca"] = comarcas_bcn[i]
    media_por_dia_comarca[i] = media_por_dia_comarca[i][nuevo_orden].reset_index(drop=True)

In [16]:
df_concatenado = pd.concat(media_por_dia_comarca).reset_index(drop=True)
df_concatenado.to_csv(output_folder + "mean_precip_x_mes.csv", index=False)

In [14]:
df_concatenado

,comarca,codi_variable,data_lectura,valor_lectura
0,Vallès Oriental,35,2009-01-01,0.000000
1,Vallès Oriental,35,2009-01-02,0.008333
2,Vallès Oriental,35,2009-01-03,0.000000
3,Vallès Oriental,35,2009-01-04,0.002083
4,Vallès Oriental,35,2009-01-05,0.002083
...,...,...,...,...
37095,Vallès Occidental,35,2023-07-05,0.007143
37096,Vallès Occidental,35,2023-07-06,0.011905
37097,Vallès Occidental,35,2023-07-07,0.000000
37098,Vallès Occidental,35,2023-07-08,0.000000
